In [ ]:
class CoffeeMachine:
    def __init__(self):
        self.resources = {
            'water': 1000,  # ml
            'milk': 500,    # ml
            'coffee': 200,   # grams
            'money': 0       # dollars
        }
        self.menu = [
            Drink("espresso", 50, 0, 18, 1.5),
            Drink("latte", 200, 150, 24, 2.5),
            Drink("cappuccino", 250, 100, 24, 3.0)
        ]
    
    def report(self):
        """Display current resources"""
        for item, amount in self.resources.items():
            unit = 'ml' if item in ['water', 'milk'] else 'g' if item == 'coffee' else '$'
            print(f"{item.capitalize()}: {amount}{unit}")
    
    def is_resource_sufficient(self, drink):
        """Check if enough resources to make drink"""
        for item in ['water', 'milk', 'coffee']:
            if getattr(drink, item) > self.resources[item]:
                print(f"Sorry there is not enough {item}.")
                return False
        return True
    
    def process_coins(self):
        """Calculate total from coins inserted"""
        print("Please insert coins.")
        quarters = int(input("How many quarters?: ")) * 0.25
        dimes = int(input("How many dimes?: ")) * 0.10
        nickels = int(input("How many nickels?: ")) * 0.05
        pennies = int(input("How many pennies?: ")) * 0.01
        return quarters + dimes + nickels + pennies
    
    def make_coffee(self, drink):
        """Deduct resources and make coffee"""
        self.resources['water'] -= drink.water
        self.resources['milk'] -= drink.milk
        self.resources['coffee'] -= drink.coffee
        self.resources['money'] += drink.cost
        print(f"Here is your {drink.name}. Enjoy!")
    
    def run(self):
        """Main machine operation"""
        while True:
            choice = input("What would you like? (espresso/latte/cappuccino): ").lower()
            
            if choice == "off":
                print("Turning off coffee machine...")
                break
            elif choice == "report":
                self.report()
            else:
                drink = next((d for d in self.menu if d.name == choice), None)
                
                if drink:
                    if self.is_resource_sufficient(drink):
                        payment = self.process_coins()
                        
                        if payment >= drink.cost:
                            change = round(payment - drink.cost, 2)
                            if change > 0:
                                print(f"Here is ${change} in change.")
                            self.make_coffee(drink)
                        else:
                            print("Sorry that's not enough money. Money refunded.")
                else:
                    print("Invalid selection. Please try again.")

class Drink:
    def __init__(self, name, water, milk, coffee, cost):
        self.name = name
        self.water = water
        self.milk = milk
        self.coffee = coffee
        self.cost = cost

# Create and run the coffee machine
machine = CoffeeMachine()
machine.run()

In [ ]:
class CoffeeMachine:
    def __init__(self):
        self.resources = {
            'water': 1000,
            'milk': 500,
            'coffee': 200,
            'money': 0
        }
        self.menu = {
            '1': Drink("Espresso", 50, 0, 18, 1.5),
            '2': Drink("Latte", 200, 150, 24, 2.5),
            '3': Drink("Cappuccino", 250, 100, 24, 3.0)
        }
        self.maintenance_mode = False
    
    def display_menu(self):
        print("\n===== MENU =====")
        for key, drink in self.menu.items():
            print(f"{key}. {drink.name} - ${drink.cost}")
        print("4. Report")
        print("5. Maintenance")
        print("6. Exit")
    
    def maintenance(self):
        """Maintenance menu for refilling and collecting money"""
        self.maintenance_mode = True
        while self.maintenance_mode:
            print("\n===== MAINTENANCE =====")
            print("1. Refill water")
            print("2. Refill milk")
            print("3. Refill coffee")
            print("4. Collect money")
            print("5. Back to main menu")
            
            choice = input("Select option: ")
            
            if choice == '1':
                amount = int(input("Enter ml of water to add: "))
                self.resources['water'] += amount
            elif choice == '2':
                amount = int(input("Enter ml of milk to add: "))
                self.resources['milk'] += amount
            elif choice == '3':
                amount = int(input("Enter grams of coffee to add: "))
                self.resources['coffee'] += amount
            elif choice == '4':
                print(f"Collected ${self.resources['money']}")
                self.resources['money'] = 0
            elif choice == '5':
                self.maintenance_mode = False
            else:
                print("Invalid option")
    
    def run(self):
        """Main machine operation"""
        print("Welcome to the Coffee Machine!")
        while True:
            self.display_menu()
            choice = input("\nEnter your choice: ")
            
            if choice == '6':
                print("Turning off coffee machine...")
                break
            elif choice == '4':
                self.report()
            elif choice == '5':
                self.maintenance()
            elif choice in self.menu:
                drink = self.menu[choice]
                if self.is_resource_sufficient(drink):
                    payment = self.process_coins()
                    if payment >= drink.cost:
                        self.make_coffee(drink)
                        if payment > drink.cost:
                            print(f"Here is ${round(payment - drink.cost, 2)} in change.")
                    else:
                        print("Sorry that's not enough money. Money refunded.")
            else:
                print("Invalid selection. Please try again.")

# Create and run the enhanced coffee machine
machine = CoffeeMachine()
machine.run()

In [ ]:
import json
import logging
from abc import ABC, abstractmethod
from datetime import datetime
from enum import Enum, auto

# Set up logging
logging.basicConfig(filename='coffee_machine.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

class DrinkType(Enum):
    ESPRESSO = auto()
    LATTE = auto()
    CAPPUCCINO = auto()
    AMERICANO = auto()
    FLAT_WHITE = auto()

class Ingredient:
    def __init__(self, name, unit, current_level, max_capacity):
        self.name = name
        self.unit = unit
        self.current_level = current_level
        self.max_capacity = max_capacity
    
    def add(self, amount):
        if self.current_level + amount > self.max_capacity:
            excess = (self.current_level + amount) - self.max_capacity
            self.current_level = self.max_capacity
            return excess
        self.current_level += amount
        return 0
    
    def use(self, amount):
        if amount > self.current_level:
            raise ValueError(f"Not enough {self.name}")
        self.current_level -= amount
    
    def __str__(self):
        return f"{self.name}: {self.current_level}{self.unit}"

class Recipe(ABC):
    def __init__(self, name, ingredients, cost):
        self.name = name
        self.ingredients = ingredients
        self.cost = cost
    
    @abstractmethod
    def prepare(self):
        pass

class EspressoRecipe(Recipe):
    def __init__(self):
        ingredients = {
            'water': 50,
            'coffee': 18
        }
        super().__init__("Espresso", ingredients, 1.5)
    
    def prepare(self):
        print("Grinding coffee beans...")
        print("Extracting espresso shot...")
        print("Espresso ready!")

class LatteRecipe(Recipe):
    def __init__(self):
        ingredients = {
            'water': 200,
            'milk': 150,
            'coffee': 24
        }
        super().__init__("Latte", ingredients, 2.5)
    
    def prepare(self):
        print("Steaming milk...")
        print("Preparing espresso...")
        print("Combining milk and espresso...")
        print("Latte ready!")

class CoffeeMachineState(ABC):
    @abstractmethod
    def insert_money(self, machine, amount):
        pass
    
    @abstractmethod
    def select_drink(self, machine, drink_type):
        pass
    
    @abstractmethod
    def dispense_drink(self, machine):
        pass
    
    @abstractmethod
    def cancel(self, machine):
        pass

class IdleState(CoffeeMachineState):
    def insert_money(self, machine, amount):
        machine.balance += amount
        machine.state = PaymentState()
        print(f"Balance: ${machine.balance:.2f}")
    
    def select_drink(self, machine, drink_type):
        print("Please insert money first")
    
    def dispense_drink(self, machine):
        print("No drink selected")
    
    def cancel(self, machine):
        print("No transaction to cancel")

class PaymentState(CoffeeMachineState):
    def insert_money(self, machine, amount):
        machine.balance += amount
        print(f"Balance: ${machine.balance:.2f}")
    
    def select_drink(self, machine, drink_type):
        recipe = machine.recipes.get(drink_type)
        if not recipe:
            print("Invalid drink selection")
            return
        
        if machine.balance < recipe.cost:
            print(f"Please insert ${recipe.cost - machine.balance:.2f} more")
            return
        
        machine.selected_recipe = recipe
        machine.state = DispensingState()
        machine.dispense_drink()
    
    def dispense_drink(self, machine):
        print("Please select a drink first")
    
    def cancel(self, machine):
        print(f"Refunding ${machine.balance:.2f}")
        machine.balance = 0
        machine.state = IdleState()

class DispensingState(CoffeeMachineState):
    def insert_money(self, machine, amount):
        print("Please wait, your drink is being prepared")
    
    def select_drink(self, machine, drink_type):
        print("Please wait, your drink is being prepared")
    
    def dispense_drink(self, machine):
        try:
            # Check and use ingredients
            for ingredient, amount in machine.selected_recipe.ingredients.items():
                machine.inventory[ingredient].use(amount)
            
            # Prepare the drink
            machine.selected_recipe.prepare()
            
            # Calculate change
            change = machine.balance - machine.selected_recipe.cost
            if change > 0:
                print(f"Returning change: ${change:.2f}")
            
            # Update machine stats
            machine.balance = 0
            machine.total_sales += machine.selected_recipe.cost
            machine.drinks_served += 1
            logging.info(f"Served {machine.selected_recipe.name} for ${machine.selected_recipe.cost:.2f}")
            
            # Return to idle
            machine.state = IdleState()
            machine.selected_recipe = None
        
        except ValueError as e:
            print(f"Error: {str(e)}")
            print("Refunding money")
            machine.balance = 0
            machine.state = IdleState()
            machine.selected_recipe = None
    
    def cancel(self, machine):
        print("Cannot cancel during dispensing")

class CoffeeMachine:
    def __init__(self):
        self.inventory = {
            'water': Ingredient("Water", "ml", 1000, 2000),
            'milk': Ingredient("Milk", "ml", 500, 1000),
            'coffee': Ingredient("Coffee", "g", 200, 500)
        }
        
        self.recipes = {
            DrinkType.ESPRESSO: EspressoRecipe(),
            DrinkType.LATTE: LatteRecipe(),
            DrinkType.CAPPUCCINO: LatteRecipe()  # Simplified for example
        }
        
        self.state = IdleState()
        self.balance = 0
        self.selected_recipe = None
        self.total_sales = 0
        self.drinks_served = 0
        self.maintenance_mode = False
    
    def insert_money(self, amount):
        self.state.insert_money(self, amount)
    
    def select_drink(self, drink_type):
        self.state.select_drink(self, drink_type)
    
    def dispense_drink(self):
        self.state.dispense_drink(self)
    
    def cancel(self):
        self.state.cancel(self)
    
    def report(self):
        print("\n=== MACHINE REPORT ===")
        for ingredient in self.inventory.values():
            print(ingredient)
        print(f"Total Sales: ${self.total_sales:.2f}")
        print(f"Drinks Served: {self.drinks_served}")
        print("=====================")
    
    def enter_maintenance(self):
        self.maintenance_mode = True
        print("\nMaintenance Mode Activated")
    
    def exit_maintenance(self):
        self.maintenance_mode = False
        print("Maintenance Mode Deactivated")
    
    def refill_ingredient(self, ingredient_name, amount):
        if not self.maintenance_mode:
            print("Must be in maintenance mode to refill")
            return
        
        if ingredient_name not in self.inventory:
            print("Invalid ingredient")
            return
        
        excess = self.inventory[ingredient_name].add(amount)
        if excess > 0:
            print(f"Warning: {excess}{self.inventory[ingredient_name].unit} of {ingredient_name} overflowed")
        else:
            print(f"{ingredient_name} refilled successfully")
    
    def save_state(self, filename):
        state = {
            'inventory': {
                name: {
                    'current': ing.current_level,
                    'max': ing.max_capacity
                }
                for name, ing in self.inventory.items()
            },
            'total_sales': self.total_sales,
            'drinks_served': self.drinks_served,
            'last_updated': datetime.now().isoformat()
        }
        
        with open(filename, 'w') as f:
            json.dump(state, f)
        
        print("Machine state saved successfully")
    
    def load_state(self, filename):
        try:
            with open(filename, 'r') as f:
                state = json.load(f)
            
            for name, data in state['inventory'].items():
                if name in self.inventory:
                    self.inventory[name].current_level = data['current']
                    self.inventory[name].max_capacity = data['max']
            
            self.total_sales = state['total_sales']
            self.drinks_served = state['drinks_served']
            print(f"Machine state loaded (last updated: {state['last_updated']})")
        
        except FileNotFoundError:
            print("No saved state found - starting with default values")

class CoffeeMachineUI:
    def __init__(self, machine):
        self.machine = machine
    
    def display_menu(self):
        print("\n==== COFFEE MACHINE ====")
        print("1. Insert Money")
        print("2. Select Drink")
        print("3. Dispense Drink")
        print("4. Cancel Transaction")
        print("5. Machine Report")
        print("6. Maintenance Menu")
        print("7. Exit")
    
    def display_drinks(self):
        print("\nAvailable Drinks:")
        for i, (drink_type, recipe) in enumerate(self.machine.recipes.items(), 1):
            print(f"{i}. {recipe.name} - ${recipe.cost:.2f}")
    
    def run(self):
        self.machine.load_state('coffee_machine_state.json')
        
        while True:
            self.display_menu()
            choice = input("Enter your choice: ")
            
            try:
                if choice == '1':
                    amount = float(input("Enter amount to insert: $"))
                    self.machine.insert_money(amount)
                
                elif choice == '2':
                    self.display_drinks()
                    drink_choice = int(input("Select drink: ")) - 1
                    drink_types = list(self.machine.recipes.keys())
                    if 0 <= drink_choice < len(drink_types):
                        self.machine.select_drink(drink_types[drink_choice])
                    else:
                        print("Invalid selection")
                
                elif choice == '3':
                    self.machine.dispense_drink()
                
                elif choice == '4':
                    self.machine.cancel()
                
                elif choice == '5':
                    self.machine.report()
                
                elif choice == '6':
                    self.maintenance_menu()
                
                elif choice == '7':
                    self.machine.save_state('coffee_machine_state.json')
                    print("Goodbye!")
                    break
                
                else:
                    print("Invalid choice")
            
            except ValueError:
                print("Please enter a valid number")
    
    def maintenance_menu(self):
        self.machine.enter_maintenance()
        
        while self.machine.maintenance_mode:
            print("\n==== MAINTENANCE ====")
            print("1. Refill Water")
            print("2. Refill Milk")
            print("3. Refill Coffee")
            print("4. View Report")
            print("5. Exit Maintenance")
            
            choice = input("Enter choice: ")
            
            if choice == '1':
                amount = int(input("Enter ml of water to add: "))
                self.machine.refill_ingredient('water', amount)
            
            elif choice == '2':
                amount = int(input("Enter ml of milk to add: "))
                self.machine.refill_ingredient('milk', amount)
            
            elif choice == '3':
                amount = int(input("Enter grams of coffee to add: "))
                self.machine.refill_ingredient('coffee', amount)
            
            elif choice == '4':
                self.machine.report()
            
            elif choice == '5':
                self.machine.exit_maintenance()
            
            else:
                print("Invalid choice")

# Create and run the coffee machine
if __name__ == "__main__":
    machine = CoffeeMachine()
    ui = CoffeeMachineUI(machine)
    ui.run()

No saved state found - starting with default values

==== COFFEE MACHINE ====
1. Insert Money
2. Select Drink
3. Dispense Drink
4. Cancel Transaction
5. Machine Report
6. Maintenance Menu
7. Exit
Invalid choice

==== COFFEE MACHINE ====
1. Insert Money
2. Select Drink
3. Dispense Drink
4. Cancel Transaction
5. Machine Report
6. Maintenance Menu
7. Exit
Invalid choice

==== COFFEE MACHINE ====
1. Insert Money
2. Select Drink
3. Dispense Drink
4. Cancel Transaction
5. Machine Report
6. Maintenance Menu
7. Exit
Invalid choice

==== COFFEE MACHINE ====
1. Insert Money
2. Select Drink
3. Dispense Drink
4. Cancel Transaction
5. Machine Report
6. Maintenance Menu
7. Exit
Invalid choice

==== COFFEE MACHINE ====
1. Insert Money
2. Select Drink
3. Dispense Drink
4. Cancel Transaction
5. Machine Report
6. Maintenance Menu
7. Exit
Invalid choice

==== COFFEE MACHINE ====
1. Insert Money
2. Select Drink
3. Dispense Drink
4. Cancel Transaction
5. Machine Report
6. Maintenance Menu
7. Exit
Invalid c

In [ ]:
import json
import logging
import hashlib
import getpass  # For secure password input
from abc import ABC, abstractmethod
from datetime import datetime
from enum import Enum, auto

# Set up logging
logging.basicConfig(filename='coffee_machine.log', level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s')

class UserRole(Enum):
    CUSTOMER = auto()
    STAFF = auto()
    ADMIN = auto()

class User:
    def __init__(self, username, password_hash, role=UserRole.CUSTOMER):
        self.username = username
        self.password_hash = password_hash
        self.role = role
    
    def verify_password(self, password):
        return self.password_hash == self._hash_password(password)
    
    @staticmethod
    def _hash_password(password):
        """Secure password hashing using SHA-256 with salt"""
        salt = "coffee_machine_salt"  # In production, use unique salt per user
        return hashlib.sha256((password + salt).encode()).hexdigest()

class AuthenticationSystem:
    def __init__(self):
        self.users = {}
        self.current_user = None
        self.load_users()
    
    def load_users(self):
        try:
            with open('users.json', 'r') as f:
                users_data = json.load(f)
                for username, data in users_data.items():
                    self.users[username] = User(
                        username=username,
                        password_hash=data['password_hash'],
                        role=UserRole[data['role']]
                    )
        except (FileNotFoundError, json.JSONDecodeError):
            # Create default admin user if no users file exists
            admin = User("admin", User._hash_password("admin123"), UserRole.ADMIN)
            self.users["admin"] = admin
            self.save_users()
    
    def save_users(self):
        users_data = {
            username: {
                'password_hash': user.password_hash,
                'role': user.role.name
            }
            for username, user in self.users.items()
        }
        with open('users.json', 'w') as f:
            json.dump(users_data, f, indent=4)
    
    def login(self):
        print("\n=== COFFEE MACHINE LOGIN ===")
        while True:
            username = input("Username: ").strip()
            if not username:
                print("Username cannot be empty")
                continue
            
            password = getpass.getpass("Password: ").strip()
            if not password:
                print("Password cannot be empty")
                continue
            
            user = self.users.get(username)
            if user and user.verify_password(password):
                self.current_user = user
                logging.info(f"User {username} logged in")
                print(f"\nWelcome, {username}!")
                return True
            
            print("Invalid username or password")
            logging.warning(f"Failed login attempt for username: {username}")
            return False
    
    def logout(self):
        if self.current_user:
            logging.info(f"User {self.current_user.username} logged out")
            self.current_user = None
        print("Logged out successfully")
    
    def create_user(self, username, password, role=UserRole.CUSTOMER):
        if username in self.users:
            raise ValueError("Username already exists")
        if not username or not password:
            raise ValueError("Username and password cannot be empty")
        
        new_user = User(username, User._hash_password(password), role)
        self.users[username] = new_user
        self.save_users()
        logging.info(f"New user created: {username} ({role.name})")
        return new_user
    
    def change_password(self, username, new_password):
        if username not in self.users:
            raise ValueError("User not found")
        if not new_password:
            raise ValueError("Password cannot be empty")
        
        self.users[username].password_hash = User._hash_password(new_password)
        self.save_users()
        logging.info(f"Password changed for user: {username}")

class DrinkType(Enum):
    ESPRESSO = auto()
    LATTE = auto()
    CAPPUCCINO = auto()
    AMERICANO = auto()
    FLAT_WHITE = auto()

class Ingredient:
    def __init__(self, name, unit, current_level, max_capacity):
        self.name = name
        self.unit = unit
        self.current_level = current_level
        self.max_capacity = max_capacity
    
    def add(self, amount):
        if amount < 0:
            raise ValueError("Amount cannot be negative")
        if self.current_level + amount > self.max_capacity:
            excess = (self.current_level + amount) - self.max_capacity
            self.current_level = self.max_capacity
            return excess
        self.current_level += amount
        return 0
    
    def use(self, amount):
        if amount < 0:
            raise ValueError("Amount cannot be negative")
        if amount > self.current_level:
            raise ValueError(f"Not enough {self.name}")
        self.current_level -= amount
    
    def __str__(self):
        return f"{self.name}: {self.current_level}{self.unit}"

class Recipe(ABC):
    def __init__(self, name, ingredients, cost):
        self.name = name
        self.ingredients = ingredients
        self.cost = cost
    
    @abstractmethod
    def prepare(self):
        pass

class EspressoRecipe(Recipe):
    def __init__(self):
        ingredients = {
            'water': 50,
            'coffee': 18
        }
        super().__init__("Espresso", ingredients, 1.5)
    
    def prepare(self):
        print("Grinding coffee beans...")
        print("Extracting espresso shot...")
        print("Espresso ready!")

class LatteRecipe(Recipe):
    def __init__(self):
        ingredients = {
            'water': 200,
            'milk': 150,
            'coffee': 24
        }
        super().__init__("Latte", ingredients, 2.5)
    
    def prepare(self):
        print("Steaming milk...")
        print("Preparing espresso...")
        print("Combining milk and espresso...")
        print("Latte ready!")

class CappuccinoRecipe(Recipe):
    def __init__(self):
        ingredients = {
            'water': 250,
            'milk': 100,
            'coffee': 24
        }
        super().__init__("Cappuccino", ingredients, 3.0)
    
    def prepare(self):
        print("Steaming milk...")
        print("Preparing espresso...")
        print("Creating milk foam...")
        print("Cappuccino ready!")

class CoffeeMachineState(ABC):
    @abstractmethod
    def insert_money(self, machine, amount):
        pass
    
    @abstractmethod
    def select_drink(self, machine, drink_type):
        pass
    
    @abstractmethod
    def dispense_drink(self, machine):
        pass
    
    @abstractmethod
    def cancel(self, machine):
        pass

class IdleState(CoffeeMachineState):
    def insert_money(self, machine, amount):
        if amount <= 0:
            print("Amount must be positive")
            return
        machine.balance += amount
        machine.state = PaymentState()
        print(f"Balance: ${machine.balance:.2f}")
    
    def select_drink(self, machine, drink_type):
        print("Please insert money first")
    
    def dispense_drink(self, machine):
        print("No drink selected")
    
    def cancel(self, machine):
        print("No transaction to cancel")

class PaymentState(CoffeeMachineState):
    def insert_money(self, machine, amount):
        if amount <= 0:
            print("Amount must be positive")
            return
        machine.balance += amount
        print(f"Balance: ${machine.balance:.2f}")
    
    def select_drink(self, machine, drink_type):
        recipe = machine.recipes.get(drink_type)
        if not recipe:
            print("Invalid drink selection")
            return
        
        if machine.balance < recipe.cost:
            print(f"Please insert ${recipe.cost - machine.balance:.2f} more")
            return
        
        machine.selected_recipe = recipe
        machine.state = DispensingState()
        machine.dispense_drink()
    
    def dispense_drink(self, machine):
        print("Please select a drink first")
    
    def cancel(self, machine):
        print(f"Refunding ${machine.balance:.2f}")
        machine.balance = 0
        machine.state = IdleState()

class DispensingState(CoffeeMachineState):
    def insert_money(self, machine, amount):
        print("Please wait, your drink is being prepared")
    
    def select_drink(self, machine, drink_type):
        print("Please wait, your drink is being prepared")
    
    def dispense_drink(self, machine):
        try:
            # Check and use ingredients
            for ingredient, amount in machine.selected_recipe.ingredients.items():
                machine.inventory[ingredient].use(amount)
            
            # Prepare the drink
            machine.selected_recipe.prepare()
            
            # Calculate change
            change = machine.balance - machine.selected_recipe.cost
            if change > 0:
                print(f"Returning change: ${change:.2f}")
            
            # Update machine stats
            machine.balance = 0
            machine.total_sales += machine.selected_recipe.cost
            machine.drinks_served += 1
            logging.info(f"Served {machine.selected_recipe.name} for ${machine.selected_recipe.cost:.2f}")
            
            # Return to idle
            machine.state = IdleState()
            machine.selected_recipe = None
        
        except ValueError as e:
            print(f"Error: {str(e)}")
            print("Refunding money")
            machine.balance = 0
            machine.state = IdleState()
            machine.selected_recipe = None
    
    def cancel(self, machine):
        print("Cannot cancel during dispensing")

class CoffeeMachine:
    def __init__(self):
        self.inventory = {
            'water': Ingredient("Water", "ml", 1000, 2000),
            'milk': Ingredient("Milk", "ml", 500, 1000),
            'coffee': Ingredient("Coffee", "g", 200, 500)
        }
        
        self.recipes = {
            DrinkType.ESPRESSO: EspressoRecipe(),
            DrinkType.LATTE: LatteRecipe(),
            DrinkType.CAPPUCCINO: CappuccinoRecipe()
        }
        
        self.state = IdleState()
        self.balance = 0
        self.selected_recipe = None
        self.total_sales = 0
        self.drinks_served = 0
        self.maintenance_mode = False
    
    def insert_money(self, amount):
        self.state.insert_money(self, amount)
    
    def select_drink(self, drink_type):
        self.state.select_drink(self, drink_type)
    
    def dispense_drink(self):
        self.state.dispense_drink(self)
    
    def cancel(self):
        self.state.cancel(self)
    
    def report(self):
        print("\n=== MACHINE REPORT ===")
        for ingredient in self.inventory.values():
            print(ingredient)
        print(f"Total Sales: ${self.total_sales:.2f}")
        print(f"Drinks Served: {self.drinks_served}")
        print("=====================")
    
    def enter_maintenance(self):
        self.maintenance_mode = True
        print("\nMaintenance Mode Activated")
    
    def exit_maintenance(self):
        self.maintenance_mode = False
        print("Maintenance Mode Deactivated")
    
    def refill_ingredient(self, ingredient_name, amount):
        if not self.maintenance_mode:
            print("Must be in maintenance mode to refill")
            return
        
        if ingredient_name not in self.inventory:
            print("Invalid ingredient")
            return
        
        try:
            excess = self.inventory[ingredient_name].add(amount)
            if excess > 0:
                print(f"Warning: {excess}{self.inventory[ingredient_name].unit} of {ingredient_name} overflowed")
            else:
                print(f"{ingredient_name} refilled successfully")
        except ValueError as e:
            print(f"Error: {str(e)}")
    
    def save_state(self, filename):
        state = {
            'inventory': {
                name: {
                    'current': ing.current_level,
                    'max': ing.max_capacity
                }
                for name, ing in self.inventory.items()
            },
            'total_sales': self.total_sales,
            'drinks_served': self.drinks_served,
            'last_updated': datetime.now().isoformat()
        }
        
        with open(filename, 'w') as f:
            json.dump(state, f, indent=4)
        
        print("Machine state saved successfully")
    
    def load_state(self, filename):
        try:
            with open(filename, 'r') as f:
                state = json.load(f)
            
            for name, data in state['inventory'].items():
                if name in self.inventory:
                    self.inventory[name].current_level = data['current']
                    self.inventory[name].max_capacity = data['max']
            
            self.total_sales = state.get('total_sales', 0)
            self.drinks_served = state.get('drinks_served', 0)
            print(f"Machine state loaded (last updated: {state.get('last_updated', 'unknown')})")
        
        except (FileNotFoundError, json.JSONDecodeError):
            print("No saved state found or invalid file - starting with default values")

class CoffeeMachineUI:
    def __init__(self, machine):
        self.machine = machine
        self.auth = AuthenticationSystem()
    
    def display_main_menu(self):
        print("\n==== MAIN MENU ====")
        print("1. Login")
        print("2. Exit")
    
    def display_drinks(self):
        print("\nAvailable Drinks:")
        for i, (drink_type, recipe) in enumerate(self.machine.recipes.items(), 1):
            print(f"{i}. {recipe.name} - ${recipe.cost:.2f}")
    
    def display_user_menu(self):
        print("\n==== COFFEE MACHINE ====")
        print("1. Insert Money")
        print("2. Select Drink")
        print("3. Dispense Drink")
        print("4. Cancel Transaction")
        
        if self.auth.current_user.role in [UserRole.STAFF, UserRole.ADMIN]:
            print("5. Machine Report")
        
        if self.auth.current_user.role == UserRole.ADMIN:
            print("6. Maintenance Menu")
            print("7. User Management")
        
        print("8. Logout")
    
    def user_management_menu(self):
        print("\n==== USER MANAGEMENT ====")
        print("1. Create New User")
        print("2. Change Password")
        print("3. List Users")
        print("4. Back to Main Menu")
    
    def maintenance_menu(self):
        print("\n==== MAINTENANCE MENU ====")
        print("1. Refill Water")
        print("2. Refill Milk")
        print("3. Refill Coffee")
        print("4. View Report")
        print("5. Exit Maintenance")
    
    def run(self):
        print("=== WELCOME TO COFFEE MACHINE ===")
        
        while True:
            if not self.auth.current_user:
                self.display_main_menu()
                choice = input("Enter your choice: ")
                
                if choice == '1':
                    if self.auth.login():
                        self.machine.load_state('coffee_machine_state.json')
                        self.user_loop()
                elif choice == '2':
                    print("Goodbye!")
                    break
                else:
                    print("Invalid choice")
            else:
                self.user_loop()
    
    def user_loop(self):
        while self.auth.current_user:
            self.display_user_menu()
            choice = input("Enter your choice: ")
            
            try:
                if choice == '1':
                    amount = float(input("Enter amount to insert: $"))
                    if amount <= 0:
                        print("Amount must be positive")
                        continue
                    self.machine.insert_money(amount)
                
                elif choice == '2':
                    self.display_drinks()
                    drink_choice = input("Select drink (or 'back' to cancel): ")
                    if drink_choice.lower() == 'back':
                        continue
                    
                    try:
                        drink_choice = int(drink_choice) - 1
                        drink_types = list(self.machine.recipes.keys())
                        if 0 <= drink_choice < len(drink_types):
                            self.machine.select_drink(drink_types[drink_choice])
                        else:
                            print("Invalid selection")
                    except ValueError:
                        print("Please enter a valid number")
                
                elif choice == '3':
                    self.machine.dispense_drink()
                
                elif choice == '4':
                    self.machine.cancel()
                
                elif choice == '5' and self.auth.current_user.role in [UserRole.STAFF, UserRole.ADMIN]:
                    self.machine.report()
                
                elif choice == '6' and self.auth.current_user.role == UserRole.ADMIN:
                    self.maintenance_loop()
                
                elif choice == '7' and self.auth.current_user.role == UserRole.ADMIN:
                    self.user_management_loop()
                
                elif choice == '8':
                    self.machine.save_state('coffee_machine_state.json')
                    self.auth.logout()
                
                else:
                    print("Invalid choice or insufficient permissions")
            
            except ValueError as e:
                print(f"Error: {str(e)}")
    
    def maintenance_loop(self):
        self.machine.enter_maintenance()
        
        while self.machine.maintenance_mode:
            self.maintenance_menu()
            choice = input("Enter choice: ")
            
            if choice == '1':
                self.handle_refill('water')
            elif choice == '2':
                self.handle_refill('milk')
            elif choice == '3':
                self.handle_refill('coffee')
            elif choice == '4':
                self.machine.report()
            elif choice == '5':
                self.machine.exit_maintenance()
            else:
                print("Invalid choice")
    
    def handle_refill(self, ingredient_name):
        try:
            amount = int(input(f"Enter amount of {ingredient_name} to add: "))
            if amount <= 0:
                print("Amount must be positive")
                return
            self.machine.refill_ingredient(ingredient_name, amount)
        except ValueError:
            print("Please enter a valid number")
    
    def user_management_loop(self):
        while True:
            self.user_management_menu()
            choice = input("Enter your choice: ")
            
            if choice == '1':
                self.handle_create_user()
            elif choice == '2':
                self.handle_change_password()
            elif choice == '3':
                self.display_user_list()
            elif choice == '4':
                break
            else:
                print("Invalid choice")
    
    def handle_create_user(self):
        username = input("Enter new username: ").strip()
        if not username:
            print("Username cannot be empty")
            return
        
        password = getpass.getpass("Enter password: ").strip()
        confirm = getpass.getpass("Confirm password: ").strip()
        
        if password != confirm:
            print("Passwords don't match!")
            return
        if not password:
            print("Password cannot be empty")
            return
        
        print("Select role:")
        print("1. Customer")
        print("2. Staff")
        print("3. Admin")
        role_choice = input("Enter role (1-3): ")
        
        roles = {
            '1': UserRole.CUSTOMER,
            '2': UserRole.STAFF,
            '3': UserRole.ADMIN
        }
        
        if role_choice in roles:
            try:
                self.auth.create_user(username, password, roles[role_choice])
                print(f"User {username} created successfully")
            except ValueError as e:
                print(str(e))
        else:
            print("Invalid role selection")
    
    def handle_change_password(self):
        username = input("Enter username: ").strip()
        if not username:
            print("Username cannot be empty")
            return
        
        new_password = getpass.getpass("Enter new password: ").strip()
        confirm = getpass.getpass("Confirm new password: ").strip()
        
        if new_password != confirm:
            print("Passwords don't match!")
            return
        if not new_password:
            print("Password cannot be empty")
            return
        
        try:
            self.auth.change_password(username, new_password)
            print("Password changed successfully")
        except ValueError as e:
            print(str(e))
    
    def display_user_list(self):
        print("\n=== USER LIST ===")
        for username, user in self.auth.users.items():
            print(f"{username} - {user.role.name}")
        print("=================")

if __name__ == "__main__":
    machine = CoffeeMachine()
    ui = CoffeeMachineUI(machine)
    ui.run()

=== WELCOME TO COFFEE MACHINE ===

==== MAIN MENU ====
1. Login
2. Exit

=== COFFEE MACHINE LOGIN ===

Welcome, admin!
No saved state found or invalid file - starting with default values

==== COFFEE MACHINE ====
1. Insert Money
2. Select Drink
3. Dispense Drink
4. Cancel Transaction
5. Machine Report
6. Maintenance Menu
7. User Management
8. Logout

Available Drinks:
1. Espresso - $1.50
2. Latte - $2.50
3. Cappuccino - $3.00
Please enter a valid number

==== COFFEE MACHINE ====
1. Insert Money
2. Select Drink
3. Dispense Drink
4. Cancel Transaction
5. Machine Report
6. Maintenance Menu
7. User Management
8. Logout
Invalid choice or insufficient permissions

==== COFFEE MACHINE ====
1. Insert Money
2. Select Drink
3. Dispense Drink
4. Cancel Transaction
5. Machine Report
6. Maintenance Menu
7. User Management
8. Logout
Invalid choice or insufficient permissions

==== COFFEE MACHINE ====
1. Insert Money
2. Select Drink
3. Dispense Drink
4. Cancel Transaction
5. Machine Report
6. Mainten

In [ ]:
import json
import logging
from abc import ABC, abstractmethod
from datetime import datetime, timedelta
from enum import Enum, auto
from typing import Dict, List, Optional, Tuple
import threading
import time
import random

# Enhanced logging setup
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('coffee_machine.log'),
        logging.StreamHandler()
    ]
)

class DrinkType(Enum):
    ESPRESSO = auto()
    LATTE = auto()
    CAPPUCCINO = auto()
    AMERICANO = auto()
    FLAT_WHITE = auto()
    MACCHIATO = auto()
    MOCHA = auto()
    HOT_WATER = auto()
    HOT_MILK = auto()

class Temperature(Enum):
    LOW = 140  # Fahrenheit
    MEDIUM = 160
    HIGH = 180
    EXTRA_HOT = 200

class Size(Enum):
    SMALL = 8  # oz
    MEDIUM = 12
    LARGE = 16

class Strength(Enum):
    MILD = 1
    MEDIUM = 2
    STRONG = 3

class Ingredient:
    def __init__(self, name: str, unit: str, current_level: float, max_capacity: float, 
                 alert_threshold: float = 0.2, expiry_days: Optional[int] = None):
        self.name = name
        self.unit = unit
        self.current_level = current_level
        self.max_capacity = max_capacity
        self.alert_threshold = alert_threshold
        self.expiry_date = datetime.now() + timedelta(days=expiry_days) if expiry_days else None
        self.last_refilled = datetime.now()
        
    def add(self, amount: float) -> float:
        if self.current_level + amount > self.max_capacity:
            excess = (self.current_level + amount) - self.max_capacity
            self.current_level = self.max_capacity
            self.last_refilled = datetime.now()
            return excess
        self.current_level += amount
        self.last_refilled = datetime.now()
        return 0
    
    def use(self, amount: float) -> None:
        if amount > self.current_level:
            raise ValueError(f"Not enough {self.name}")
        self.current_level -= amount
    
    def is_low(self) -> bool:
        return self.current_level < (self.max_capacity * self.alert_threshold)
    
    def is_expired(self) -> bool:
        if self.expiry_date:
            return datetime.now() > self.expiry_date
        return False
    
    def __str__(self) -> str:
        status = f"{self.name}: {self.current_level:.1f}{self.unit}/{self.max_capacity}{self.unit}"
        if self.is_low():
            status += " (LOW)"
        if self.is_expired():
            status += " (EXPIRED)"
        return status

class Recipe(ABC):
    def __init__(self, name: str, ingredients: Dict[str, float], cost: float, 
                 prep_time: int = 30, default_temp: Temperature = Temperature.MEDIUM,
                 default_size: Size = Size.MEDIUM):
        self.name = name
        self.ingredients = ingredients
        self.cost = cost
        self.prep_time = prep_time  # seconds
        self.default_temp = default_temp
        self.default_size = default_size
        
    @abstractmethod
    def prepare(self, temperature: Temperature, size: Size, strength: Strength) -> None:
        pass
    
    def get_adjusted_ingredients(self, size: Size) -> Dict[str, float]:
        size_factor = size.value / self.default_size.value
        return {ing: amount * size_factor for ing, amount in self.ingredients.items()}

class EspressoRecipe(Recipe):
    def __init__(self):
        ingredients = {
            'water': 50,
            'coffee': 18
        }
        super().__init__("Espresso", ingredients, 1.5, 25, Temperature.HIGH, Size.SMALL)
    
    def prepare(self, temperature: Temperature, size: Size, strength: Strength) -> None:
        print(f"Grinding coffee beans for {strength.name.lower()} strength...")
        print(f"Heating water to {temperature.value}°F...")
        print("Extracting espresso shot...")
        time.sleep(self.prep_time * (strength.value / 2))  # Simulate preparation time
        print("Espresso ready!")

class LatteRecipe(Recipe):
    def __init__(self):
        ingredients = {
            'water': 50,
            'milk': 150,
            'coffee': 18
        }
        super().__init__("Latte", ingredients, 2.5, 40, Temperature.MEDIUM, Size.MEDIUM)
    
    def prepare(self, temperature: Temperature, size: Size, strength: Strength) -> None:
        print(f"Grinding coffee beans for {strength.name.lower()} strength...")
        print(f"Steaming milk to {temperature.value}°F...")
        print("Preparing espresso...")
        time.sleep(self.prep_time * (strength.value / 2))
        print("Combining milk and espresso...")
        print("Latte ready!")

class CustomRecipe(Recipe):
    def __init__(self, name: str, ingredients: Dict[str, float], cost: float):
        super().__init__(name, ingredients, cost)
    
    def prepare(self, temperature: Temperature, size: Size, strength: Strength) -> None:
        print(f"Preparing custom drink '{self.name}'...")
        print(f"Size: {size.name}, Temp: {temperature.value}°F, Strength: {strength.name}")
        time.sleep(self.prep_time)
        print(f"{self.name} ready!")

class CoffeeMachineState(ABC):
    @abstractmethod
    def insert_money(self, machine, amount: float) -> None:
        pass
    
    @abstractmethod
    def select_drink(self, machine, drink_type: DrinkType, 
                    temperature: Temperature, size: Size, strength: Strength) -> None:
        pass
    
    @abstractmethod
    def dispense_drink(self, machine) -> None:
        pass
    
    @abstractmethod
    def cancel(self, machine) -> None:
        pass
    
    @abstractmethod
    def maintenance(self, machine) -> None:
        pass

class IdleState(CoffeeMachineState):
    def insert_money(self, machine, amount: float) -> None:
        machine.balance += amount
        machine.state = PaymentState()
        print(f"Balance: ${machine.balance:.2f}")
        logging.info(f"Money inserted: ${amount:.2f}")
    
    def select_drink(self, machine, drink_type: DrinkType, 
                    temperature: Temperature, size: Size, strength: Strength) -> None:
        print("Please insert money first")
    
    def dispense_drink(self, machine) -> None:
        print("No drink selected")
    
    def cancel(self, machine) -> None:
        print("No transaction to cancel")
    
    def maintenance(self, machine) -> None:
        machine.enter_maintenance()

class PaymentState(CoffeeMachineState):
    def insert_money(self, machine, amount: float) -> None:
        machine.balance += amount
        print(f"Balance: ${machine.balance:.2f}")
        logging.info(f"Additional money inserted: ${amount:.2f}")
    
    def select_drink(self, machine, drink_type: DrinkType, 
                    temperature: Temperature, size: Size, strength: Strength) -> None:
        recipe = machine.get_recipe(drink_type, size)
        if not recipe:
            print("Invalid drink selection")
            return
        
        if machine.balance < recipe.cost:
            print(f"Please insert ${recipe.cost - machine.balance:.2f} more")
            return
        
        machine.selected_recipe = recipe
        machine.selected_temp = temperature
        machine.selected_size = size
        machine.selected_strength = strength
        machine.state = DispensingState()
        machine.dispense_drink()
    
    def dispense_drink(self, machine) -> None:
        print("Please select a drink first")
    
    def cancel(self, machine) -> None:
        print(f"Refunding ${machine.balance:.2f}")
        logging.info(f"Transaction canceled. Refunded: ${machine.balance:.2f}")
        machine.balance = 0
        machine.state = IdleState()
    
    def maintenance(self, machine) -> None:
        print("Please cancel transaction first")

class DispensingState(CoffeeMachineState):
    def insert_money(self, machine, amount: float) -> None:
        print("Please wait, your drink is being prepared")
    
    def select_drink(self, machine, drink_type: DrinkType, 
                    temperature: Temperature, size: Size, strength: Strength) -> None:
        print("Please wait, your drink is being prepared")
    
    def dispense_drink(self, machine) -> None:
        try:
            # Check and use ingredients
            adjusted_ingredients = machine.selected_recipe.get_adjusted_ingredients(machine.selected_size)
            for ingredient, amount in adjusted_ingredients.items():
                machine.inventory[ingredient].use(amount)
            
            # Prepare the drink in a separate thread
            def prepare_drink():
                machine.selected_recipe.prepare(
                    machine.selected_temp, 
                    machine.selected_size, 
                    machine.selected_strength
                )
                # Calculate change
                change = machine.balance - machine.selected_recipe.cost
                if change > 0:
                    print(f"Returning change: ${change:.2f}")
                
                # Update machine stats
                machine.balance = 0
                machine.total_sales += machine.selected_recipe.cost
                machine.drinks_served += 1
                logging.info(
                    f"Served {machine.selected_recipe.name} (Size: {machine.selected_size.name}, "
                    f"Temp: {machine.selected_temp.value}°F) for ${machine.selected_recipe.cost:.2f}"
                )
                
                # Return to idle
                machine.state = IdleState()
                machine.selected_recipe = None
            
            # Start drink preparation thread
            preparation_thread = threading.Thread(target=prepare_drink)
            preparation_thread.start()
        
        except ValueError as e:
            print(f"Error: {str(e)}")
            print("Refunding money")
            logging.error(f"Dispensing error: {str(e)}")
            machine.balance = 0
            machine.state = IdleState()
            machine.selected_recipe = None
    
    def cancel(self, machine) -> None:
        print("Cannot cancel during dispensing")
    
    def maintenance(self, machine) -> None:
        print("Cannot enter maintenance during dispensing")

class CleaningState(CoffeeMachineState):
    def insert_money(self, machine, amount: float) -> None:
        print("Machine is being cleaned. Please wait.")
    
    def select_drink(self, machine, drink_type: DrinkType, 
                    temperature: Temperature, size: Size, strength: Strength) -> None:
        print("Machine is being cleaned. Please wait.")
    
    def dispense_drink(self, machine) -> None:
        print("Machine is being cleaned. Please wait.")
    
    def cancel(self, machine) -> None:
        print("No transaction to cancel during cleaning")
    
    def maintenance(self, machine) -> None:
        print("Already in maintenance mode")

class CoffeeMachine:
    def __init__(self):
        self.inventory = {
            'water': Ingredient("Water", "ml", 1000, 2000),
            'milk': Ingredient("Milk", "ml", 500, 1000, expiry_days=7),
            'coffee': Ingredient("Coffee", "g", 200, 500),
            'sugar': Ingredient("Sugar", "g", 300, 500),
            'cocoa': Ingredient("Cocoa", "g", 100, 200, expiry_days=30),
            'vanilla': Ingredient("Vanilla", "ml", 50, 100, expiry_days=60)
        }
        
        self.recipes = self._initialize_recipes()
        self.custom_recipes: List[Recipe] = []
        
        self.state: CoffeeMachineState = IdleState()
        self.balance: float = 0
        self.selected_recipe: Optional[Recipe] = None
        self.selected_temp: Optional[Temperature] = None
        self.selected_size: Optional[Size] = None
        self.selected_strength: Optional[Strength] = None
        
        self.total_sales: float = 0
        self.drinks_served: int = 0
        self.maintenance_mode: bool = False
        self.cleaning_required: bool = False
        self.cleaning_cycles: int = 0
        self.last_cleaned: Optional[datetime] = None
        
        # Start background monitoring thread
        self.monitor_thread = threading.Thread(target=self._monitor_machine, daemon=True)
        self.monitor_thread.start()
    
    def _initialize_recipes(self) -> Dict[DrinkType, Recipe]:
        return {
            DrinkType.ESPRESSO: EspressoRecipe(),
            DrinkType.LATTE: LatteRecipe(),
            DrinkType.CAPPUCCINO: LatteRecipe(),  # Simplified for example
            DrinkType.AMERICANO: CustomRecipe("Americano", {'water': 200, 'coffee': 18}, 2.0),
            DrinkType.FLAT_WHITE: CustomRecipe("Flat White", {'water': 50, 'milk': 100, 'coffee': 18}, 2.5),
            DrinkType.MACCHIATO: CustomRecipe("Macchiato", {'water': 30, 'milk': 30, 'coffee': 18}, 2.0),
            DrinkType.MOCHA: CustomRecipe("Mocha", {'water': 50, 'milk': 100, 'coffee': 18, 'cocoa': 10}, 3.0),
            DrinkType.HOT_WATER: CustomRecipe("Hot Water", {'water': 250}, 0.5),
            DrinkType.HOT_MILK: CustomRecipe("Hot Milk", {'milk': 250}, 1.5)
        }
    
    def _monitor_machine(self) -> None:
        """Background thread to monitor machine status"""
        while True:
            time.sleep(60)  # Check every minute
            
            # Check for low ingredients
            low_ingredients = [ing.name for ing in self.inventory.values() if ing.is_low()]
            if low_ingredients:
                logging.warning(f"Low ingredients: {', '.join(low_ingredients)}")
            
            # Check for expired ingredients
            expired_ingredients = [ing.name for ing in self.inventory.values() if ing.is_expired()]
            if expired_ingredients:
                logging.error(f"Expired ingredients: {', '.join(expired_ingredients)}")
            
            # Check if cleaning is needed (after every 10 drinks)
            if self.drinks_served > 0 and self.drinks_served % 10 == 0 and not self.cleaning_required:
                self.cleaning_required = True
                logging.warning("Machine needs cleaning")
    
    def get_recipe(self, drink_type: DrinkType, size: Size) -> Optional[Recipe]:
        """Get recipe adjusted for size"""
        recipe = self.recipes.get(drink_type)
        if not recipe:
            return None
        
        # Create a temporary adjusted recipe
        adjusted_recipe = CustomRecipe(
            recipe.name,
            recipe.get_adjusted_ingredients(size),
            recipe.cost * (size.value / recipe.default_size.value)
        )
        return adjusted_recipe
    
    def insert_money(self, amount: float) -> None:
        if amount <= 0:
            print("Amount must be positive")
            return
        self.state.insert_money(self, amount)
    
    def select_drink(self, drink_type: DrinkType, 
                    temperature: Temperature = Temperature.MEDIUM,
                    size: Size = Size.MEDIUM,
                    strength: Strength = Strength.MEDIUM) -> None:
        if self.cleaning_required and not self.maintenance_mode:
            print("Machine needs cleaning. Please contact staff.")
            return
        self.state.select_drink(self, drink_type, temperature, size, strength)
    
    def dispense_drink(self) -> None:
        self.state.dispense_drink(self)
    
    def cancel(self) -> None:
        self.state.cancel(self)
    
    def report(self) -> None:
        print("\n=== MACHINE REPORT ===")
        print(f"Status: {'Maintenance' if self.maintenance_mode else 'Operational'}")
        print(f"Cleaning Needed: {'Yes' if self.cleaning_required else 'No'}")
        print(f"Last Cleaned: {self.last_cleaned or 'Never'}")
        
        print("\nIngredients:")
        for ingredient in self.inventory.values():
            print(f"  {ingredient}")
            if ingredient.last_refilled:
                print(f"    Last refilled: {ingredient.last_refilled.strftime('%Y-%m-%d')}")
            if ingredient.expiry_date:
                print(f"    Expires: {ingredient.expiry_date.strftime('%Y-%m-%d')}")
        
        print("\nFinancial:")
        print(f"Total Sales: ${self.total_sales:.2f}")
        print(f"Drinks Served: {self.drinks_served}")
        print("=====================")
        logging.info("Machine report generated")
    
    def enter_maintenance(self) -> None:
        if isinstance(self.state, DispensingState):
            print("Cannot enter maintenance during dispensing")
            return
        
        self.maintenance_mode = True
        self.state = CleaningState()
        print("\nMaintenance Mode Activated")
        logging.info("Entered maintenance mode")
    
    def exit_maintenance(self) -> None:
        self.maintenance_mode = False
        self.state = IdleState()
        print("Maintenance Mode Deactivated")
        logging.info("Exited maintenance mode")
    
    def refill_ingredient(self, ingredient_name: str, amount: float) -> None:
        if not self.maintenance_mode:
            print("Must be in maintenance mode to refill")
            return
        
        if ingredient_name not in self.inventory:
            print("Invalid ingredient")
            return
        
        excess = self.inventory[ingredient_name].add(amount)
        if excess > 0:
            print(f"Warning: {excess}{self.inventory[ingredient_name].unit} of {ingredient_name} overflowed")
        else:
            print(f"{ingredient_name} refilled successfully")
        logging.info(f"Refilled {ingredient_name} with {amount}{self.inventory[ingredient_name].unit}")
    
    def clean_machine(self) -> None:
        if not self.maintenance_mode:
            print("Must be in maintenance mode to clean")
            return
        
        print("Starting cleaning cycle...")
        time.sleep(10)  # Simulate cleaning time
        self.cleaning_required = False
        self.cleaning_cycles += 1
        self.last_cleaned = datetime.now()
        print("Cleaning complete!")
        logging.info("Machine cleaning completed")
    
    def add_custom_recipe(self, name: str, ingredients: Dict[str, float], cost: float) -> None:
        if not self.maintenance_mode:
            print("Must be in maintenance mode to add recipes")
            return
        
        # Validate ingredients
        for ing in ingredients:
            if ing not in self.inventory:
                print(f"Invalid ingredient: {ing}")
                return
        
        new_recipe = CustomRecipe(name, ingredients, cost)
        self.custom_recipes.append(new_recipe)
        print(f"Custom recipe '{name}' added successfully")
        logging.info(f"Added custom recipe: {name}")
    
    def save_state(self, filename: str) -> None:
        state = {
            'inventory': {
                name: {
                    'current': ing.current_level,
                    'max': ing.max_capacity,
                    'last_refilled': ing.last_refilled.isoformat(),
                    'expiry_date': ing.expiry_date.isoformat() if ing.expiry_date else None
                }
                for name, ing in self.inventory.items()
            },
            'total_sales': self.total_sales,
            'drinks_served': self.drinks_served,
            'cleaning_cycles': self.cleaning_cycles,
            'last_cleaned': self.last_cleaned.isoformat() if self.last_cleaned else None,
            'last_updated': datetime.now().isoformat(),
            'custom_recipes': [
                {
                    'name': recipe.name,
                    'ingredients': recipe.ingredients,
                    'cost': recipe.cost
                }
                for recipe in self.custom_recipes
            ]
        }
        
        with open(filename, 'w') as f:
            json.dump(state, f, indent=2)
        
        print("Machine state saved successfully")
        logging.info("Machine state saved")
    
    def load_state(self, filename: str) -> None:
        try:
            with open(filename, 'r') as f:
                state = json.load(f)
            
            for name, data in state['inventory'].items():
                if name in self.inventory:
                    self.inventory[name].current_level = data['current']
                    self.inventory[name].max_capacity = data['max']
                    self.inventory[name].last_refilled = datetime.fromisoformat(data['last_refilled'])
                    if data['expiry_date']:
                        self.inventory[name].expiry_date = datetime.fromisoformat(data['expiry_date'])
            
            self.total_sales = state['total_sales']
            self.drinks_served = state['drinks_served']
            self.cleaning_cycles = state['cleaning_cycles']
            self.last_cleaned = datetime.fromisoformat(state['last_cleaned']) if state['last_cleaned'] else None
            
            # Load custom recipes
            self.custom_recipes = []
            for recipe_data in state.get('custom_recipes', []):
                self.custom_recipes.append(
                    CustomRecipe(
                        recipe_data['name'],
                        recipe_data['ingredients'],
                        recipe_data['cost']
                    )
                )
            
            print(f"Machine state loaded (last updated: {state['last_updated']})")
            logging.info("Machine state loaded")
        
        except FileNotFoundError:
            print("No saved state found - starting with default values")
        except Exception as e:
            print(f"Error loading state: {str(e)}")
            logging.error(f"Error loading state: {str(e)}")

class CoffeeMachineUI:
    def __init__(self, machine: CoffeeMachine):
        self.machine = machine
        self.running = True
    
    def display_menu(self) -> None:
        print("\n==== COFFEE MACHINE ====")
        print("1. Insert Money")
        print("2. Select Drink")
        print("3. Dispense Drink")
        print("4. Cancel Transaction")
        print("5. Machine Report")
        print("6. Maintenance Menu")
        print("7. Exit")
    
    def display_drinks(self) -> None:
        print("\nAvailable Drinks:")
        for i, (drink_type, recipe) in enumerate(self.machine.recipes.items(), 1):
            print(f"{i}. {recipe.name} - ${recipe.cost:.2f}")
        
        if self.machine.custom_recipes:
            print("\nCustom Drinks:")
            for i, recipe in enumerate(self.machine.custom_recipes, len(self.machine.recipes)+1):
                print(f"{i}. {recipe.name} - ${recipe.cost:.2f}")
    
    def display_options(self) -> None:
        print("\nDrink Options:")
        print("1. Temperature:")
        for i, temp in enumerate(Temperature, 1):
            print(f"   {i}. {temp.name} ({temp.value}°F)")
        
        print("2. Size:")
        for i, size in enumerate(Size, 1):
            print(f"   {i}. {size.name} ({size.value}oz)")
        
        print("3. Strength:")
        for i, strength in enumerate(Strength, 1):
            print(f"   {i}. {strength.name}")
    
    def run(self) -> None:
        self.machine.load_state('coffee_machine_state.json')
        
        while self.running:
            self.display_menu()
            choice = input("Enter your choice: ")
            
            try:
                if choice == '1':
                    amount = float(input("Enter amount to insert: $"))
                    self.machine.insert_money(amount)
                
                elif choice == '2':
                    self.display_drinks()
                    drink_choice = int(input("Select drink: ")) - 1
                    
                    # Combine standard and custom recipes
                    drink_types = list(self.machine.recipes.keys())
                    all_recipes = drink_types + self.machine.custom_recipes
                    
                    if 0 <= drink_choice < len(all_recipes):
                        self.display_options()
                        
                        # Get temperature
                        temp_choice = int(input("Select temperature (1-4): ")) - 1
                        temperature = list(Temperature)[temp_choice] if 0 <= temp_choice < len(Temperature) else Temperature.MEDIUM
                        
                        # Get size
                        size_choice = int(input("Select size (1-3): ")) - 1
                        size = list(Size)[size_choice] if 0 <= size_choice < len(Size) else Size.MEDIUM
                        
                        # Get strength
                        strength_choice = int(input("Select strength (1-3): ")) - 1
                        strength = list(Strength)[strength_choice] if 0 <= strength_choice < len(Strength) else Strength.MEDIUM
                        
                        if drink_choice < len(drink_types):
                            self.machine.select_drink(drink_types[drink_choice], temperature, size, strength)
                        else:
                            # For custom recipes, use a temporary DrinkType
                            temp_type = DrinkType.HOT_WATER  # Using as placeholder
                            custom_recipe = self.machine.custom_recipes[drink_choice - len(drink_types)]
                            self.machine.recipes[temp_type] = custom_recipe
                            self.machine.select_drink(temp_type, temperature, size, strength)
                    else:
                        print("Invalid selection")
                
                elif choice == '3':
                    self.machine.dispense_drink()
                
                elif choice == '4':
                    self.machine.cancel()
                
                elif choice == '5':
                    self.machine.report()
                
                elif choice == '6':
                    self.maintenance_menu()
                
                elif choice == '7':
                    self.machine.save_state('coffee_machine_state.json')
                    print("Goodbye!")
                    self.running = False
                
                else:
                    print("Invalid choice")
            
            except ValueError as e:
                print(f"Invalid input: {str(e)}")
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                logging.error(f"UI error: {str(e)}")
    
    def maintenance_menu(self) -> None:
        self.machine.enter_maintenance()
        
        while self.machine.maintenance_mode:
            print("\n==== MAINTENANCE ====")
            print("1. Refill Ingredients")
            print("2. Clean Machine")
            print("3. Add Custom Recipe")
            print("4. View Report")
            print("5. Exit Maintenance")
            
            choice = input("Enter choice: ")
            
            try:
                if choice == '1':
                    print("\nAvailable Ingredients:")
                    for i, (name, ing) in enumerate(self.machine.inventory.items(), 1):
                        print(f"{i}. {ing}")
                    
                    ing_choice = int(input("Select ingredient to refill: ")) - 1
                    ingredient_names = list(self.machine.inventory.keys())
                    if 0 <= ing_choice < len(ingredient_names):
                        amount = float(input(f"Enter amount to add ({self.machine.inventory[ingredient_names[ing_choice]].unit}): "))
                        self.machine.refill_ingredient(ingredient_names[ing_choice], amount)
                    else:
                        print("Invalid selection")
                
                elif choice == '2':
                    if self.machine.cleaning_required:
                        print("Starting cleaning cycle...")
                        self.machine.clean_machine()
                    else:
                        print("Machine doesn't need cleaning yet")
                
                elif choice == '3':
                    name = input("Enter recipe name: ")
                    ingredients = {}
                    print("\nAvailable Ingredients:")
                    for i, (name, ing) in enumerate(self.machine.inventory.items(), 1):
                        print(f"{i}. {name} ({ing.unit})")
                    
                    while True:
                        ing_choice = input("Add ingredient (number) or 'done' to finish: ")
                        if ing_choice.lower() == 'done':
                            break
                        
                        try:
                            ing_num = int(ing_choice) - 1
                            if 0 <= ing_num < len(self.machine.inventory):
                                ing_name = list(self.machine.inventory.keys())[ing_num]
                                amount = float(input(f"Enter amount of {ing_name} ({self.machine.inventory[ing_name].unit}): "))
                                ingredients[ing_name] = amount
                            else:
                                print("Invalid number")
                        except ValueError:
                            print("Please enter a number or 'done'")
                    
                    if not ingredients:
                        print("No ingredients added")
                        continue
                    
                    cost = float(input("Enter drink cost: $"))
                    self.machine.add_custom_recipe(name, ingredients, cost)
                
                elif choice == '4':
                    self.machine.report()
                
                elif choice == '5':
                    self.machine.exit_maintenance()
                
                else:
                    print("Invalid choice")
            
            except ValueError as e:
                print(f"Invalid input: {str(e)}")
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                logging.error(f"Maintenance menu error: {str(e)}")

if __name__ == "__main__":
    try:
        machine = CoffeeMachine()
        ui = CoffeeMachineUI(machine)
        ui.run()
    except Exception as e:
        logging.critical(f"Fatal error: {str(e)}")
        print(f"A critical error occurred: {str(e)}")